In [45]:
from data_collection import (store_database_for_eys_gene,
                             parse_lovd,
                             LOVD_PATH,
                             set_lovd_dtypes,
                             filter_eys_genes)

import re

In [46]:
store_database_for_eys_gene("lovd", override=False)

The file at ../data/lovd/lovd_data.txt already exists.


In [47]:
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")
for i in data:
    print(i)
    display(data[i])

Genes


,id,name,chromosome,chrom_band,imprinting,refseq_genomic,refseq_UD,reference,url_homepage,url_external,...,header,header_align,footer,footer_align,created_by,created_date,edited_by,edited_date,updated_by,updated_date
0,EYS,eyes shut homolog (Drosophila),6,q12,unknown,NG_023443.2,UD_132085377375,,http://www.LOVD.nl/EYS,,...,"<font color=\""#FF0000\"">This database is one o...",-1,,-1,00001,2012-02-13 00:00:00,00006,2023-08-30 13:08:19,00000,2024-04-19 20:27:30


Transcripts


,id,geneid,name,id_mutalyzer,id_ncbi,id_ensembl,id_protein_ncbi,id_protein_ensembl,id_protein_uniprot,remarks,position_c_mrna_start,position_c_mrna_end,position_c_cds_end,position_g_mrna_start,position_g_mrna_end,created_by,created_date,edited_by,edited_date
0,00007329,EYS,transcript variant 1,001,NM_001142800.1,,NP_001136272.1,,,,-538,10051,9435,66417118,64429876,,0000-00-00 00:00:00,,


Diseases


...

,id,allele,effectid,chromosome,position_g_start,position_g_end,type,average_frequency,owned_by,VariantOnGenome/DBID,...,VariantOnGenome/Genetic_origin,VariantOnGenome/Segregation,VariantOnGenome/dbSNP,VariantOnGenome/VIP,VariantOnGenome/Methylation,VariantOnGenome/ISCN,VariantOnGenome/DNA/hg38,VariantOnGenome/ClinVar,VariantOnGenome/ClinicalClassification,VariantOnGenome/ClinicalClassification/Method
0,0000036426,3,50,6,64498971,64498971,subst,0.000742922,00552,EYS_000007,...,Germline,,,0,,,g.63789078A>G,,VUS,
1,0000059881,3,55,6,65655758,65655758,subst,0.00115297,00229,EYS_000001,...,Germline,yes,,0,,,g.64945865T>G,,VUS,
2,0000059883,1,11,6,65336143,65336143,subst,0.224189,00229,EYS_000002,...,Germline,no,,0,,,g.64626250G>A,,benign,
3,0000059884,1,15,6,65300869,65300869,subst,0.000837928,00229,EYS_000003,...,Germline,,,0,,,g.64590976G>A,,benign,
4,0000059885,1,11,6,65016998,65016999,del,0,00229,EYS_000004,...,Germline,yes,,0,,,g.64307105_64307106del,,benign,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2536,0000964211,0,30,6,65767634,65767634,subst,0.243022,02330,EYS_000248,...,CLASSIFICATION record,,,,,,,,likely benign,
2537,0000964212,0,30,6,65767643,65767643,del,0,02330,EYS_000926,...,CLASSIFICATION record,,,,,,,,likely benign,
2538,0000964215,0,50,6,66005927,66005927,subst,0.000112112,02327,EYS_000253,...,CLASSIFICATION record,,,,,,,,VUS,
2539,0000964216,0,50,6,66044874,66044874,subst,0.0000818974,02327,EYS_000256,...,CLASSIFICATION record,,,,,,,,VUS,


Variants_On_Transcripts


,id,transcriptid,effectid,position_c_start,position_c_start_intron,position_c_end,position_c_end_intron,VariantOnTranscript/DNA,VariantOnTranscript/RNA,VariantOnTranscript/Protein,VariantOnTranscript/Exon
0,0000036426,00007329,50,7558,0,7558,0,c.7558T>C,r.(?),p.(Phe2520Leu),38
1,0000059881,00007329,55,2309,0,2309,0,c.2309A>C,r.(?),p.(Gln770Pro),15
2,0000059883,00007329,11,3444,-5,3444,-5,c.3444-5C>T,r.(?),p.(=),22i
3,0000059884,00007329,15,4891,0,4891,0,c.4891C>T,r.(?),p.(Pro1631Ser),26
4,0000059885,00007329,11,6079,-4,6079,-3,c.6079-4_6079-3del,r.(?),p.(=),29i
...,...,...,...,...,...,...,...,...,...,...,...
2536,0000964211,00007329,30,2024,-14,2024,-14,c.2024-14C>T,r.(=),p.(=),
2537,0000964212,00007329,30,2024,-15,2024,-15,c.2024-15del,r.(=),p.(=),
2538,0000964215,00007329,50,1852,0,1852,0,c.1852G>A,r.(?),p.(Gly618Ser),
2539,0000964216,00007329,50,1765,0,1765,0,c.1765A>G,r.(?),p.(Arg589Gly),


Screenings_To_Variants


,screeningid,variantid
0,0000000126,0000783293
1,0000000211,0000790459
2,0000001640,0000235838
3,0000016557,0000036426
4,0000033164,0000059884
...,...,...
2144,0000449279,0000959046
2145,0000449284,0000959051
2146,0000449293,0000959060
2147,0000449295,0000959474


In [48]:
set_lovd_dtypes(data)
for i in data:
    print(i)
    display(data[i].info())

Genes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               1 non-null      string        
 1   name             1 non-null      string        
 2   chromosome       1 non-null      Int64         
 3   chrom_band       1 non-null      string        
 4   imprinting       1 non-null      string        
 5   refseq_genomic   1 non-null      string        
 6   refseq_UD        1 non-null      string        
 7   reference        1 non-null      string        
 8   url_homepage     1 non-null      string        
 9   url_external     1 non-null      string        
 10  allow_download   1 non-null      bool          
 11  id_hgnc          1 non-null      Int64         
 12  id_entrez        1 non-null      Int64         
 13  id_omim          1 non-null      Int64         
 14  show_hgmd        1 non-null      bool   

None

Transcripts
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     1 non-null      Int64         
 1   geneid                 1 non-null      string        
 2   name                   1 non-null      string        
 3   id_mutalyzer           1 non-null      Int64         
 4   id_ncbi                1 non-null      string        
 5   id_ensembl             1 non-null      string        
 6   id_protein_ncbi        1 non-null      string        
 7   id_protein_ensembl     1 non-null      string        
 8   id_protein_uniprot     1 non-null      string        
 9   remarks                1 non-null      string        
 10  position_c_mrna_start  1 non-null      Int64         
 11  position_c_mrna_end    1 non-null      Int64         
 12  position_c_cds_end     1 non-null      Int64         
 1

None

Diseases
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            15 non-null     Int64         
 1   symbol        15 non-null     string        
 2   name          15 non-null     string        
 3   inheritance   15 non-null     string        
 4   id_omim       4 non-null      Int64         
 5   tissues       15 non-null     string        
 6   features      15 non-null     string        
 7   remarks       15 non-null     string        
 8   created_by    15 non-null     Int64         
 9   created_date  15 non-null     datetime64[ns]
 10  edited_by     11 non-null     Int64         
 11  edited_date   11 non-null     datetime64[ns]
dtypes: Int64(4), datetime64[ns](2), string(6)
memory usage: 1.6 KB


...

None

Variants_On_Transcripts
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2541 entries, 0 to 2540
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           2541 non-null   Int64 
 1   transcriptid                 2541 non-null   Int64 
 2   effectid                     2541 non-null   Int64 
 3   position_c_start             2540 non-null   Int64 
 4   position_c_start_intron      2541 non-null   Int64 
 5   position_c_end               2540 non-null   Int64 
 6   position_c_end_intron        2541 non-null   Int64 
 7   VariantOnTranscript/DNA      2541 non-null   string
 8   VariantOnTranscript/RNA      2541 non-null   string
 9   VariantOnTranscript/Protein  2541 non-null   string
 10  VariantOnTranscript/Exon     2541 non-null   string
dtypes: Int64(7), string(4)
memory usage: 235.9 KB


None

Screenings_To_Variants
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   screeningid  2149 non-null   Int64
 1   variantid    2149 non-null   Int64
dtypes: Int64(2)
memory usage: 37.9 KB


None

In [49]:
import pandas as pd
from data_collection import CLINVAR_PATH
from data_collection import store_database_for_eys_gene
store_database_for_eys_gene("clinvar", override=False)


clinvar_data = pd.read_csv(CLINVAR_PATH + "/clinvar_data.txt", sep='\t')

File already exists


In [50]:
clinvar_data

,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,Germline classification,Germline date last evaluated,Germline review status,Somatic clinical impact,Somatic clinical impact date last evaluated,Somatic clinical impact review status,Oncogenicity classification,Oncogenicity date last evaluated,Oncogenicity review status,Unnamed: 24
0,GRCh38/hg38 6p12.1-q12(chr6:53931543-68149750)x3,BAG2|BEND6|BMP5|COL21A1|DST|DST-AS1|ERVH-3|EYS...,NaN,See cases,VCV000148002,6,53796341 - 68859642,6,53931543 - 68149750,148002,...,Pathogenic,"Jan 1, 2010",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GRCh38/hg38 6p11.2-q12(chr6:57466921-68712228)x3,ADGRB3|ADGRB3-DT|ERVH-3|EYS|FKBP1C|KHDRBS2|LGS...,NaN,See cases,VCV000148991,6,57329882 - 69422120,6,57466921 - 68712228,148991,...,Pathogenic,"Jan 1, 2012",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GRCh38/hg38 6q12(chr6:63719407-64994324)x3,EYS|LOC113175011|LOC113175012|LOC123744835|LOC...,NaN,See cases,VCV000148291,6,64429303 - 65704217,6,63719407 - 64994324,148291,...,Likely benign,"Jan 1, 2011",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NM_001370348.2(PHF3):c.*6334T>C,EYS|PHF3,NaN,Retinitis pigmentosa,VCV000910562,6,64429938,6,63720042,910562,...,Uncertain significance,"Jan 1, 2018","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NM_001370348.2(PHF3):c.*6340C>T,EYS|PHF3,NaN,Retinitis pigmentosa,VCV000910563,6,64429944,6,63720048,910563,...,Uncertain significance,"Jan 1, 2018","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4657,NM_001142800.1(EYS):c.(6078+1_6079-1)_(6191+1_...,EYS,NaN,Retinitis pigmentosa,VCV000636135,NaN,NaN,NaN,NaN,636135,...,Pathogenic,"Jan 1, 2018",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4658,NM_001142800.1(EYS):c.(2641+1_2642-1)_(2846+1_...,EYS,NaN,Retinitis pigmentosa,VCV000636134,NaN,NaN,NaN,NaN,636134,...,Likely pathogenic,"Jan 1, 2018",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4659,NM_001142800.1:c.(2137+1_2138-1)_(2259+1_2260-...,EYS,NaN,Retinitis pigmentosa,VCV000636133,NaN,NaN,NaN,NaN,636133,...,Likely pathogenic,"Jan 1, 2018",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4660,NM_001142800.1:c.(2023+1_2024-1)_(3443+1_3444-...,EYS,NaN,Retinal dystrophy,VCV000636132,NaN,NaN,NaN,NaN,636132,...,Pathogenic,"Jan 1, 2018",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Extracting the g. locations from the lovd data:

In [51]:
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")
g_locations = []
for key, value in data["Variants_On_Genome"]["VariantOnGenome/DNA/hg38"].items():
    print(key, value)
    stripped = re.search(r'g\.(\d+)', value)
    if stripped:
        g_locations.append(stripped.group(1))
    else:
        g_locations.append("")
print(g_locations)

0 g.63789078A>G
1 g.64945865T>G
2 g.64626250G>A
3 g.64590976G>A
4 g.64307105_64307106del
5 g.63778108T>C
6 g.65495379dup
7 g.65495379dup
8 g.65353453G>A
9 g.63864227C>G
10 g.64307024C>T
11 g.64590397T>G
12 g.64945794G>A
13 
14 g.64902422_64902438del
15 g.64902422_64902438del
16 g.64840707_64997105del
17 g.64840707_64997105del
18 g.64840707_64997105del
19 g.65295915del
20 g.65295915del
21 g.65295915del
22 g.65057728_65320715del
23 g.64436244C>A
24 g.65057728_65320715del
25 g.64436244C>A
26 g.65057728_65320715del
27 g.64436244C>A
28 g.63762613C>T
29 g.63762613C>T
30 g.63720626A>T
31 g.63720626A>T
32 g.63720626A>T
33 g.63720626A>T
34 g.63720626A>T
35 g.64066349del
36 g.65384425del
37 g.64230600C>T
38 g.64230600C>T
39 g.63726645C>A
40 g.63720737_63720746del
41 g.63720737_63720746del
42 g.64230600C>T
43 g.64081870C>T
44 g.64230600C>T
45 g.64081870C>T
46 g.64230600C>T
47 g.64081870C>T
48 g.64230600C>T
49 g.64081870C>T
50 g.65334981T>C
51 g.63788163G>C
52 g.64813377C>T
53 g.63864209C>T
54 g.6

Adding the g. locations to the lovd data at the end:

In [52]:
print(g_locations)

data["Variants_On_Genome"]["GLocations"] = g_locations

data["Variants_On_Genome"]


['63789078', '64945865', '64626250', '64590976', '64307105', '63778108', '65495379', '65495379', '65353453', '63864227', '64307024', '64590397', '64945794', '', '64902422', '64902422', '64840707', '64840707', '64840707', '65295915', '65295915', '65295915', '65057728', '64436244', '65057728', '64436244', '65057728', '64436244', '63762613', '63762613', '63720626', '63720626', '63720626', '63720626', '63720626', '64066349', '65384425', '64230600', '64230600', '63726645', '63720737', '63720737', '64230600', '64081870', '64230600', '64081870', '64230600', '64081870', '64230600', '64081870', '65334981', '63788163', '64813377', '63864209', '63721602', '63720900', '63778094', '63721602', '63762478', '65384383', '65335072', '64388791', '64066431', '63864331', '64822812', '', '64439165', '64439165', '64626122', '65335104', '65296034', '65405253', '65495130', '63721682', '64590616', '64590608', '64590616', '64590608', '65494867', '', '65494867', '', '65494988', '63720919', '63720668', '63720919',

,id,allele,effectid,chromosome,position_g_start,position_g_end,type,average_frequency,owned_by,VariantOnGenome/DBID,...,VariantOnGenome/Segregation,VariantOnGenome/dbSNP,VariantOnGenome/VIP,VariantOnGenome/Methylation,VariantOnGenome/ISCN,VariantOnGenome/DNA/hg38,VariantOnGenome/ClinVar,VariantOnGenome/ClinicalClassification,VariantOnGenome/ClinicalClassification/Method,GLocations
0,0000036426,3,50,6,64498971,64498971,subst,0.000742922,00552,EYS_000007,...,,,0,,,g.63789078A>G,,VUS,,63789078
1,0000059881,3,55,6,65655758,65655758,subst,0.00115297,00229,EYS_000001,...,yes,,0,,,g.64945865T>G,,VUS,,64945865
2,0000059883,1,11,6,65336143,65336143,subst,0.224189,00229,EYS_000002,...,no,,0,,,g.64626250G>A,,benign,,64626250
3,0000059884,1,15,6,65300869,65300869,subst,0.000837928,00229,EYS_000003,...,,,0,,,g.64590976G>A,,benign,,64590976
4,0000059885,1,11,6,65016998,65016999,del,0,00229,EYS_000004,...,yes,,0,,,g.64307105_64307106del,,benign,,64307105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2536,0000964211,0,30,6,65767634,65767634,subst,0.243022,02330,EYS_000248,...,,,,,,,,likely benign,,
2537,0000964212,0,30,6,65767643,65767643,del,0,02330,EYS_000926,...,,,,,,,,likely benign,,
2538,0000964215,0,50,6,66005927,66005927,subst,0.000112112,02327,EYS_000253,...,,,,,,,,VUS,,
2539,0000964216,0,50,6,66044874,66044874,subst,0.0000818974,02327,EYS_000256,...,,,,,,,,VUS,,


merged data from clinvar and lovd databases on the g. locations:

In [53]:
clinvar_data_g_merged = pd.merge(clinvar_data, data["Variants_On_Genome"], left_on="GRCh38Location", right_on="GLocations")

clinvar_data_g_merged

,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,VariantOnGenome/Segregation,VariantOnGenome/dbSNP,VariantOnGenome/VIP,VariantOnGenome/Methylation,VariantOnGenome/ISCN,VariantOnGenome/DNA/hg38,VariantOnGenome/ClinVar,VariantOnGenome/ClinicalClassification,VariantOnGenome/ClinicalClassification/Method,GLocations
0,NM_001142800.2(EYS):c.9414T>G (p.Asp3138Glu),EYS|PHF3,"D3138E, D3159E",Retinal dystrophy|not provided,VCV000964717,6,64430513,6,63720617,964717,...,,rs374161234,0,,,g.63720617A>C,,VUS,,63720617
1,NM_001142800.2(EYS):c.9414T>G (p.Asp3138Glu),EYS|PHF3,"D3138E, D3159E",Retinal dystrophy|not provided,VCV000964717,6,64430513,6,63720617,964717,...,,,0,,,g.63720617A>C,,likely pathogenic,,63720617
2,NM_001142800.2(EYS):c.9405T>C (p.Tyr3135=),EYS|PHF3,NaN,not provided,VCV001626869,6,64430522,6,63720626,1626869,...,yes,,0,,,g.63720626A>T,,pathogenic,,63720626
3,NM_001142800.2(EYS):c.9405T>C (p.Tyr3135=),EYS|PHF3,NaN,not provided,VCV001626869,6,64430522,6,63720626,1626869,...,yes,,0,,,g.63720626A>T,,pathogenic,,63720626
4,NM_001142800.2(EYS):c.9405T>C (p.Tyr3135=),EYS|PHF3,NaN,not provided,VCV001626869,6,64430522,6,63720626,1626869,...,yes,,0,,,g.63720626A>T,,pathogenic,,63720626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,NM_001142800.2(EYS):c.-448+5G>A,EYS,NaN,Retinitis pigmentosa 25|Retinal dystrophy|not ...,VCV000865902,6,66417023,6,65707130,865902,...,,,0,,,g.65707130C>T,,VUS,,65707130
1909,NM_001142800.2(EYS):c.-448+5G>A,EYS,NaN,Retinitis pigmentosa 25|Retinal dystrophy|not ...,VCV000865902,6,66417023,6,65707130,865902,...,,,0,,,g.65707130C>T,,VUS,ACMG,65707130
1910,NM_001142800.2(EYS):c.-459C>T,EYS,NaN,not provided|Retinitis pigmentosa|Retinitis pi...,VCV000357760,6,66417039,6,65707146,357760,...,,,0,,,g.65707146G>A,,VUS,,65707146
1911,NM_001142800.2(EYS):c.-459C>T,EYS,NaN,not provided|Retinitis pigmentosa|Retinitis pi...,VCV000357760,6,66417039,6,65707146,357760,...,,,0,,,g.65707146G>A,,VUS,,65707146


Data converted from clinvar name to simplified gene name:

In [54]:
filtered_data = filter_eys_genes(clinvar_data)

clinvar_data["Simple_Gene"] = filtered_data

print(filtered_data)

['', '', '', '', '', '', '', '', '', '', '', '', 'c.9434A>G', 'c.9433T>G', 'c.9432A>G', '', '', 'c.9427G>A', 'c.9423T>C', 'c.9420A>C', 'c.9417A>G', '', 'c.9414T>G', 'c.9408T>C', 'c.9405T>C', '', 'c.9405T>A', 'c.9403T>C', '', 'c.9401T>C', '', '', '', 'c.9399T>C', '', 'c.9396C>T', 'c.9392G>A', 'c.9392G>C', '', 'c.9388G>T', '', '', 'c.9381T>G', 'c.9379A>G', '', 'c.9378A>T', 'c.9372T>G', 'c.9371T>C', 'c.9369C>T', '', 'c.9368A>C', 'c.9366A>G', '', '', 'c.9354T>C', 'c.9352T>C', '', 'c.9348A>G', '', 'c.9346G>C', 'c.9345T>C', 'c.9344T>A', '', 'c.9338A>G', 'c.9336T>C', '', '', 'c.9334A>G', 'c.9329G>A', 'c.9329G>C', 'c.9327T>C', '', 'c.9324T>C', 'c.9321T>C', '', 'c.9313A>T', '', 'c.9304G>A', 'c.9303A>G', '', '', 'c.9301C>T', 'c.9300T>C', 'c.9300T>G', '', 'c.9294C>T', '', 'c.9291T>C', 'c.9288A>G', '', 'c.9285G>A', 'c.9280A>G', 'c.9279T>C', '', 'c.9276T>C', '', 'c.9272A>C', 'c.9269T>C', '', '', 'c.9263G>A', '', '', 'c.9260T>C', 'c.9258T>C', 'c.9258T>A', 'c.9252T>C', '', 'c.9249C>T', 'c.9248G>A', '

Merging the clinvar data with the lovd data on the simplified gene name:

In [58]:
clinvar_data_merged = pd.merge(clinvar_data, data["Variants_On_Transcripts"], left_on="Simple_Gene", right_on="VariantOnTranscript/DNA")

clinvar_data_merged

,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,transcriptid,effectid,position_c_start,position_c_start_intron,position_c_end,position_c_end_intron,VariantOnTranscript/DNA,VariantOnTranscript/RNA,VariantOnTranscript/Protein,VariantOnTranscript/Exon
0,NM_001142800.2(EYS):c.9414T>G (p.Asp3138Glu),EYS|PHF3,"D3138E, D3159E",Retinal dystrophy|not provided,VCV000964717,6,64430513,6,63720617,964717,...,00007329,50,9414,0,9414,0,c.9414T>G,r.(?),p.(Asp3138Glu),
1,NM_001142800.2(EYS):c.9414T>G (p.Asp3138Glu),EYS|PHF3,"D3138E, D3159E",Retinal dystrophy|not provided,VCV000964717,6,64430513,6,63720617,964717,...,00007329,70,9414,0,9414,0,c.9414T>G,r.(?),p.(Asp3138Glu),
2,NM_001142800.2(EYS):c.9405T>A (p.Tyr3135Ter),EYS|PHF3,"Y3156*, Y3135*",EYS-related condition|Retinitis pigmentosa|Ret...,VCV000000538,6,64430522,6,63720626,538,...,00007329,99,9405,0,9405,0,c.9405T>A,r.(?),p.(Tyr3135*),43
3,NM_001142800.2(EYS):c.9405T>A (p.Tyr3135Ter),EYS|PHF3,"Y3156*, Y3135*",EYS-related condition|Retinitis pigmentosa|Ret...,VCV000000538,6,64430522,6,63720626,538,...,00007329,99,9405,0,9405,0,c.9405T>A,r.(?),p.(Tyr3135*),43
4,NM_001142800.2(EYS):c.9405T>A (p.Tyr3135Ter),EYS|PHF3,"Y3156*, Y3135*",EYS-related condition|Retinitis pigmentosa|Ret...,VCV000000538,6,64430522,6,63720626,538,...,00007329,99,9405,0,9405,0,c.9405T>A,r.(?),p.(Tyr3135*),43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,NM_001142800.2(EYS):c.35T>C (p.Met12Thr),EYS,M12T,Retinitis pigmentosa 25|Retinal dystrophy|not ...,VCV000872090,6,66205269,6,65495376,872090,...,00007329,70,35,0,35,0,c.35T>C,r.(?),p.(Met12Thr),
1277,NM_001142800.2(EYS):c.35T>C (p.Met12Thr),EYS,M12T,Retinitis pigmentosa 25|Retinal dystrophy|not ...,VCV000872090,6,66205269,6,65495376,872090,...,00007329,70,35,0,35,0,c.35T>C,r.(?),p.(Met12Thr),
1278,NM_001142800.2(EYS):c.19G>A (p.Val7Ile),EYS,V7I,not provided,VCV002163247,6,66205285,6,65495392,2163247,...,00007329,30,19,0,19,0,c.19G>A,r.(?),p.(Val7Ile),
1279,NM_001142800.2(EYS):c.16A>G (p.Ile6Val),EYS,I6V,not provided,VCV001503187,6,66205288,6,65495395,1503187,...,00007329,30,16,0,16,0,c.16A>G,r.(?),p.(Ile6Val),


In [57]:
pd.merge(clinvar_data_merged, data["Variants_On_Genome"], on="id")

,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,VariantOnGenome/Segregation,VariantOnGenome/dbSNP,VariantOnGenome/VIP,VariantOnGenome/Methylation,VariantOnGenome/ISCN,VariantOnGenome/DNA/hg38,VariantOnGenome/ClinVar,VariantOnGenome/ClinicalClassification,VariantOnGenome/ClinicalClassification/Method,GLocations
0,NM_001142800.2(EYS):c.9414T>G (p.Asp3138Glu),EYS|PHF3,"D3138E, D3159E",Retinal dystrophy|not provided,VCV000964717,6,64430513,6,63720617,964717,...,,rs374161234,0,,,g.63720617A>C,,VUS,,63720617
1,NM_001142800.2(EYS):c.9414T>G (p.Asp3138Glu),EYS|PHF3,"D3138E, D3159E",Retinal dystrophy|not provided,VCV000964717,6,64430513,6,63720617,964717,...,,,0,,,g.63720617A>C,,likely pathogenic,,63720617
2,NM_001142800.2(EYS):c.9405T>A (p.Tyr3135Ter),EYS|PHF3,"Y3156*, Y3135*",EYS-related condition|Retinitis pigmentosa|Ret...,VCV000000538,6,64430522,6,63720626,538,...,yes,,0,,,g.63720626A>T,,pathogenic,,63720626
3,NM_001142800.2(EYS):c.9405T>A (p.Tyr3135Ter),EYS|PHF3,"Y3156*, Y3135*",EYS-related condition|Retinitis pigmentosa|Ret...,VCV000000538,6,64430522,6,63720626,538,...,yes,,0,,,g.63720626A>T,,pathogenic,,63720626
4,NM_001142800.2(EYS):c.9405T>A (p.Tyr3135Ter),EYS|PHF3,"Y3156*, Y3135*",EYS-related condition|Retinitis pigmentosa|Ret...,VCV000000538,6,64430522,6,63720626,538,...,yes,,0,,,g.63720626A>T,,pathogenic,,63720626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,NM_001142800.2(EYS):c.35T>C (p.Met12Thr),EYS,M12T,Retinitis pigmentosa 25|Retinal dystrophy|not ...,VCV000872090,6,66205269,6,65495376,872090,...,,,0,,,g.65495376A>G,,likely pathogenic,,65495376
1277,NM_001142800.2(EYS):c.35T>C (p.Met12Thr),EYS,M12T,Retinitis pigmentosa 25|Retinal dystrophy|not ...,VCV000872090,6,66205269,6,65495376,872090,...,?,,0,,,g.65495376A>G,,likely pathogenic,,65495376
1278,NM_001142800.2(EYS):c.19G>A (p.Val7Ile),EYS,V7I,not provided,VCV002163247,6,66205285,6,65495392,2163247,...,,,,,,g.65495392C>T,,likely benign,,65495392
1279,NM_001142800.2(EYS):c.16A>G (p.Ile6Val),EYS,I6V,not provided,VCV001503187,6,66205288,6,65495395,1503187,...,,,0,,,g.65495395T>C,,likely benign,,65495395
